In [91]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Indexing and Selecting Data

In this section we will focus on slicing, dicing and getting the data we want out of our pandas objects, namely `Series` and `DataFrame` objects.

## Different Choices for indexing

Pandas has never landed on a single best strategy for indexing and filtering data. As such there are a few ways to accomplish the same goal. And, all things being equal you should choose the syntax that fits best in your data workflow, makes the most sense to you, and fits in with standard practices in your field. Currently, pandas supports 4 types of indexing.

* `.loc` & `.iloc`
* Boolean Indexing
* `where` method and masking
* `query()` method


### `.loc` & `.iloc`

* `.loc` is label based -- ie you subset based on the *name* of the index
  * it will raise a `KeyError` when the item(s) are not found
  * `loc` will accept:
    - A single label e.g `5` or `a` (Note that 5 is interpreted as a *label* of the index. This is **not** an integer position along the index).
    - a list or array of labels `['a', 'b', 'c']`
    - A slice object with start and stop labels `'a':'f'`
    - A `callable` function
* `iloc` is integer position based but may also be used with a boolean array.
  * It will raise a `IndexError` if a requested indexer is out-of-bounds
  * `.iloc` will accept:
    - An integer e.g. `5`
    - A list or array of integers
    - a slice object of integers `1:7`
    - A boolean array
    - A calable functions

> `iloc` and `loc` will work identically for a time series specific index unlkess otherwise specified

In [92]:
import pandas as pd
import numpy as np

dates = pd.date_range('1/1/2000', periods=8)
df = pd.DataFrame(np.random.randn(8, 4),
                  index=dates, columns=['A', 'B', 'C', 'D'])

df
df.loc['2000-01-04':]
df.iloc[0]
df.iloc[0:2]
df.loc[lambda df: df['A'] < 0]


,A,B,C,D
2000-01-01,-1.013628,-0.786148,-0.425696,0.387059
2000-01-02,-0.903192,0.918178,0.306110,1.230266
2000-01-03,-1.007050,0.989362,-1.776510,0.237550
2000-01-04,0.005526,-0.989675,1.927623,1.758136
2000-01-05,-0.853957,-1.167425,1.761221,1.733904
2000-01-06,0.732855,-0.049241,0.044860,-0.582032
2000-01-07,0.176099,0.963616,-1.083304,0.306015
2000-01-08,0.567815,1.683493,1.694295,-0.561078


,A,B,C,D
2000-01-04,0.005526,-0.989675,1.927623,1.758136
2000-01-05,-0.853957,-1.167425,1.761221,1.733904
2000-01-06,0.732855,-0.049241,0.044860,-0.582032
2000-01-07,0.176099,0.963616,-1.083304,0.306015
2000-01-08,0.567815,1.683493,1.694295,-0.561078


A   -1.013628
B   -0.786148
C   -0.425696
D    0.387059
Name: 2000-01-01 00:00:00, dtype: float64

,A,B,C,D
2000-01-01,-1.013628,-0.786148,-0.425696,0.387059
2000-01-02,-0.903192,0.918178,0.306110,1.230266


,A,B,C,D
2000-01-01,-1.013628,-0.786148,-0.425696,0.387059
2000-01-02,-0.903192,0.918178,0.306110,1.230266
2000-01-03,-1.007050,0.989362,-1.776510,0.237550
2000-01-05,-0.853957,-1.167425,1.761221,1.733904


It is possible to combine `iloc` and `loc` however these mroe advanced features are outside the scope of this guide. For more information, see the [documentation](https://pandas.pydata.org/docs/user_guide/indexing.html) 

### Boolean Indexing

Another common operation is the use of boolean vectors to filter the data. The operators are: `|` for `or`, `&` for `and`, and `~` for `not`. These must be grouped by using parentheses, since by default Python will evaluate an expression such as `df['A'] > 2 & df['B'] < 3` as `df['A'] > (2 & df['B']) < 3`, while the desired evaluation order is `(df['A'] > 2) & (df['B'] < 3)`.


In [93]:
df[(df['A'] > 0.5) & (df['D'] < 0)]

s = df['A']
s[(s > 1)]

,A,B,C,D
2000-01-06,0.732855,-0.049241,0.044860,-0.582032
2000-01-08,0.567815,1.683493,1.694295,-0.561078


Series([], Freq: D, Name: A, dtype: float64)

### `pandas.Series.isin()`

The `isin()` method of `Series` returns a boolean vector that is true whenever the `Series` element exists in the passed list. This allows you to select rows where one or more columns have spcific values you want.


In [94]:
s = pd.Series(np.arange(5), dtype='int64')
s
s.isin([2,4,6])

s[s.isin([2,4,6])]

0    0
1    1
2    2
3    3
4    4
dtype: int64

0    False
1    False
2     True
3    False
4     True
dtype: bool

2    2
4    4
dtype: int64

This is also handy for string columns in a `DataFrame`

In [95]:
df = pd.DataFrame({
    'a': ['Alice', 'Bob', 'Charlie', 'Dan']
})

df[df['a'].isin(['Alice', 'Bob'])]

,a
0,Alice
1,Bob


### `where` method and masking

Other selection methods will typically return a subset of your original data. To guarantee that your output has the same shape as your original data, you can use the `where` method for `Series` or    `DataFrame` objects. 

Contrast these two methods:

In [96]:
s[s>0]

s.where(s > 0)

1    1
2    2
3    3
4    4
dtype: int64

0    NaN
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In addition, `where` takes an optional `other` argument for replacement of values where the condition is False. This makes for a cleaner, and faster implementation than a classic `if else` statement.

In [97]:
df = pd.DataFrame(np.random.randn(6,4))
df
df.where(df < 0, -df) #Converts all positives to negatives
df.where(df < 0) # equivalient to df.where(df < 0, np.Nan)
df.where(df < 0, 'FOO') # This will coerce the columns to object dtype

,0,1,2,3
0,-1.499337,-0.712993,1.773219,0.853786
1,0.499003,-0.044362,-0.042380,-0.060451
2,0.523542,1.283749,-0.216277,-0.414309
3,0.116036,-1.455056,0.862467,-1.011314
4,-1.321772,-0.487437,-0.813700,1.185390
5,-0.195453,0.846110,-0.068692,0.070656


,0,1,2,3
0,-1.499337,-0.712993,-1.773219,-0.853786
1,-0.499003,-0.044362,-0.042380,-0.060451
2,-0.523542,-1.283749,-0.216277,-0.414309
3,-0.116036,-1.455056,-0.862467,-1.011314
4,-1.321772,-0.487437,-0.813700,-1.185390
5,-0.195453,-0.846110,-0.068692,-0.070656


,0,1,2,3
0,-1.499337,-0.712993,NaN,NaN
1,NaN,-0.044362,-0.042380,-0.060451
2,NaN,NaN,-0.216277,-0.414309
3,NaN,-1.455056,NaN,-1.011314
4,-1.321772,-0.487437,-0.813700,NaN
5,-0.195453,NaN,-0.068692,NaN


,0,1,2,3
0,-1.499337,-0.712993,FOO,FOO
1,FOO,-0.044362,-0.04238,-0.060451
2,FOO,FOO,-0.216277,-0.414309
3,FOO,-1.455056,FOO,-1.011314
4,-1.321772,-0.487437,-0.8137,FOO
5,-0.195453,FOO,-0.068692,FOO


An Important note. Boolean indexing (the `[]` operator) uses `where()` under the hood. This means the following statements are equivalent.

In [98]:
df.where(df < 0)
df[df < 0]

,0,1,2,3
0,-1.499337,-0.712993,NaN,NaN
1,NaN,-0.044362,-0.042380,-0.060451
2,NaN,NaN,-0.216277,-0.414309
3,NaN,-1.455056,NaN,-1.011314
4,-1.321772,-0.487437,-0.813700,NaN
5,-0.195453,NaN,-0.068692,NaN


,0,1,2,3
0,-1.499337,-0.712993,NaN,NaN
1,NaN,-0.044362,-0.042380,-0.060451
2,NaN,NaN,-0.216277,-0.414309
3,NaN,-1.455056,NaN,-1.011314
4,-1.321772,-0.487437,-0.813700,NaN
5,-0.195453,NaN,-0.068692,NaN


Now, you can see that replacing or setting values based on some criteria can be done intuitively like so.

In [99]:
df[df < 0] = 0
df

,0,1,2,3
0,0.000000,0.000000,1.773219,0.853786
1,0.499003,0.000000,0.000000,0.000000
2,0.523542,1.283749,0.000000,0.000000
3,0.116036,0.000000,0.862467,0.000000
4,0.000000,0.000000,0.000000,1.185390
5,0.000000,0.846110,0.000000,0.070656


### `query()` method

Dataframe objects have a `query()` method that allows you to select data using an expresison. 

In [100]:
df = pd.DataFrame(np.random.randn(8,3), columns=['a', 'b', 'c'])
df

# pure python
df[(df['a'] < df['b']) & (df['b'] < df['c'])]

#query
df.query("(a < b) & (b < c)")

#equivalent
df.query("(a < b) and (b < c)")

#equivalent
df.query("a < b < c")

,a,b,c
0,-0.011958,0.338907,0.427632
1,-0.738879,0.649017,2.318080
2,1.586060,-0.516915,0.353535
3,0.640441,-0.563947,-1.166829
4,0.255683,-1.042812,-0.609902
5,-0.736196,0.005125,2.150602
6,-1.035094,0.714584,0.080856
7,-0.807396,-0.153863,-0.846238


,a,b,c
0,-0.011958,0.338907,0.427632
1,-0.738879,0.649017,2.318080
5,-0.736196,0.005125,2.150602


,a,b,c
0,-0.011958,0.338907,0.427632
1,-0.738879,0.649017,2.318080
5,-0.736196,0.005125,2.150602


,a,b,c
0,-0.011958,0.338907,0.427632
1,-0.738879,0.649017,2.318080
5,-0.736196,0.005125,2.150602


,a,b,c
0,-0.011958,0.338907,0.427632
1,-0.738879,0.649017,2.318080
5,-0.736196,0.005125,2.150602


`query()` also supports the `in` and `not in` operators

In [101]:
df = pd.DataFrame({
    'a': list('aabbccddeeff'),
    'b': list('aaaabbbbcccc'),
    'c': np.random.randint(5, size=12),
    'd': np.random.randint(9, size=12)
})
df

# pure python
df[df['a'].isin(df['b'])]

# query
df.query('a in b')

# pure python
df[~df['a'].isin(df['b'])]

# query
df.query('a not in b')

,a,b,c,d
0,a,a,1,8
1,a,a,0,0
2,b,a,1,8
3,b,a,1,8
4,c,b,3,2
5,c,b,4,2
6,d,b,1,0
7,d,b,0,3
8,e,c,0,2
9,e,c,1,3


,a,b,c,d
0,a,a,1,8
1,a,a,0,0
2,b,a,1,8
3,b,a,1,8
4,c,b,3,2
5,c,b,4,2


,a,b,c,d
0,a,a,1,8
1,a,a,0,0
2,b,a,1,8
3,b,a,1,8
4,c,b,3,2
5,c,b,4,2


,a,b,c,d
6,d,b,1,0
7,d,b,0,3
8,e,c,0,2
9,e,c,1,3
10,f,c,1,0
11,f,c,0,5


,a,b,c,d
6,d,b,1,0
7,d,b,0,3
8,e,c,0,2
9,e,c,1,3
10,f,c,1,0
11,f,c,0,5


`query` allows you to write very succint expressions and fits nicely within piped operations. In addition, the engine that evaluates these expressions has been shown to be faster and more performant that other methods, making it a reliable and simple filtering tool.

### Duplicate Data

If you want to identify and remove duplicate rows in a DataFrame, there are two methods that will help: `duplicated` and `drop_duplicates`. Each takes as an argument the columns to use to identify duplicated rows.

* `duplicated` returns a boolean vector whose length is the number of rows, and which indiciates whether a row is duplicated
* `drop_duplicates` effectively runs `duplicated` and then drops rows that return True

By default, the first observed row of a duplicate set is retained, however this can be modified using the `keep` argument. 

In [102]:
df = pd.DataFrame({
    'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
    'b': ['x','y', 'x', 'y', 'x', 'x', 'x'],
    'c': np.random.randn(7)
})

df.duplicated('a')

df.drop_duplicates('a')
df.drop_duplicates('a', keep=False) # retains no unique values of duplicated rows

0    False
1     True
2    False
3     True
4     True
5    False
6    False
dtype: bool

,a,b,c
0,one,x,-0.438985
2,two,x,1.774196
5,three,x,0.530597
6,four,x,0.707356


,a,b,c
5,three,x,0.530597
6,four,x,0.707356


To remove fully duplicated rows, pass no column names to `drop_duplicates`. This will likely be a common operation in your data workflows.

In [103]:
df = pd.DataFrame({
    'a': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    'b': ['x','y', 'x', 'y', 'x', 'x', 'x']
})

df.drop_duplicates()

,a,b
0,one,x
1,one,y
3,two,y
4,two,x


### Missing Data

Dealing with missing data can be headache. It can also be one of the most important parts of good data analysis. We will cover missing data extensively in a later chatper. However, for now we will introduce the `dropna` function. This will serve as a quick means to remove missing data on a subset of your columns.

Both `DatFrame` and `Series` have a `dropna()` method. However, the `DataFrame.dropna()` method has considerably more options and so will typically be the standard use method. 


In [104]:
df = pd.DataFrame({
    'a': [1,2,3,4,5,np.NaN, 7, np.NaN],
    'b': ['A', 'B', None, None, 'E', 'F', 'G', None]
})

# drop all rows with an empty value
df.dropna()

# drop rows where a is empty
df.dropna(subset='a')

# drop rows where entire row is empty
df.dropna(how = 'all')

# equivalent
df.dropna(subset=['a', 'b'], how = 'all')

,a,b
0,1.0,A
1,2.0,B
4,5.0,E
6,7.0,G


,a,b
0,1.0,A
1,2.0,B
2,3.0,None
3,4.0,None
4,5.0,E
6,7.0,G


,a,b
0,1.0,A
1,2.0,B
2,3.0,None
3,4.0,None
4,5.0,E
5,NaN,F
6,7.0,G


,a,b
0,1.0,A
1,2.0,B
2,3.0,None
3,4.0,None
4,5.0,E
5,NaN,F
6,7.0,G
